In [1]:
import os, glob, re, io, random, gensim, smart_open, logging
import numpy as np

from PyPDF2 import PdfReader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from nltk.tokenize import word_tokenize


In [2]:
pdfReaders = []
pdfFiles = []
docLabels = []

# taggedData = []
# wordSets = []

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored"
txtExtractDir = "/Users/tillman/t-root/dev/projects/2022/pdf-correlator/gitignored/txt-extractions/"


In [3]:
# read files and extract to .txt files
def read_files():
    os.chdir(rootDir)
    for file in glob.glob("**/*.pdf"):
        pdfFiles.append(file)
        pdfReaders.append(PdfReader(file))
    print("pdf files read")
        
def extract_to_txt():
    os.chdir(txtExtractDir)
    for i in pdfReaders:
        with open(i.metadata.title + ".txt", 'w', encoding="utf-8") as file:
            
            # add doc title to array for tagging
            docLabels.append(i.metadata.title)
            print("doc labels: " + str(len(docLabels)))
            
            for j in range(0, len(i.pages)):
                # create a text file for future reference / future use
                file.write(i.getPage(j).extract_text())                   
                
    print("text extraction complete")
    
    
# def clean_tag_tokenize():
    
#     # read from the generated .txt files, clean and append to array
#     for i in range (0, len(docLabels)):
#         words = open(docLabels[i] + '.txt').read()
#         words = words.replace("\n", " ")
#         words = words.replace("  ", " ")
#         words = words.lower()
#         words = ''.join([i for i in words if i.isalpha() or i.isspace() or (i in '.!?:"')])
#         words = words.replace(".", " . ")
#         words = words.replace("!", " ! ")
#         words = words.replace("?", " ? ")
#         words = words.replace(":", " : ")
#         words = words.replace('"', ' " ')
#         words = words.split()
        
#         wordSets.append(words)
#         print("word sets: " + str(len(wordSets)))
        
#     # tag the cleaned words with txt document heading (derived from pdf doc title)
#     for i in range (0, len(docLabels)):
#         taggedData.append(TaggedDocument(words=wordSets[i], tags=docLabels[i]))
#         print(taggedData[0])
        
def read_corpus(fname, tokens_only=False):
        
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
#                 for label in docLabels:
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])



    
    

In [4]:
read_files()

pdf files read


In [5]:
extract_to_txt()

doc labels: 1
doc labels: 2
text extraction complete


In [6]:
# clean_tag_tokenize()
# got this function from a tutorial. While it works, it's superseeded by read_corpus()
# which is from the official gensim tutorials and includes a built in txt cleaning function

In [7]:
# creating a singular corpus, from multiple documents. Does that work...?
trainDocs = []
for i in range(len(docLabels)):
    trainDocs.append(txtExtractDir + docLabels[i] + ".txt")

trainCorpus = []
for i in range(len(trainDocs)):
    trainCorpus += list(read_corpus(trainDocs[i])) 

    
print(trainCorpus)

[TaggedDocument(words=['new', 'dark', 'agethis', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagieseke', 'gmail', 'com', 'on', 'new', 'dark', 'the', 'end', 'of', 'the', 'futurejames', 'bridle'], tags=[0]), TaggedDocument(words=['this', 'ebook', 'is', 'licensed', 'to', 'lena', 'gieseke', 'lenagieseke', 'gmail', 'com', 'on', 'this', 'paperback', 'edition', 'first', 'published', 'by', 'verso', 'first', 'published', 'by', 'verso', 'james', 'bridle', 'all', 'rights', 'reservedthe', 'moral', 'rights', 'of', 'the', 'author', 'have', 'been', 'asserted', 'versouk', 'meard', 'street', 'london', 'egus', 'jay', 'street', 'suite', 'brooklyn', 'ny', 'versobooks', 'comverso', 'is', 'the', 'imprint', 'of', 'new', 'left', 'booksisbn', 'isbn', 'uk', 'ebk', 'isbn', 'us', 'ebk', 'british', 'library', 'cataloguing', 'in', 'publication', 'dataa', 'catalogue', 'record', 'for', 'this', 'book', 'isavailable', 'from', 'the', 'british', 'librarylibrary', 'of', 'congress', 'cataloging', 'in', 'publicat

In [8]:
# train the model

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(trainCorpus)


2022-09-22 18:11:31,661 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>', 'datetime': '2022-09-22T18:11:31.661245', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'created'}
2022-09-22 18:11:31,661 : INFO : collecting all words and their counts
2022-09-22 18:11:31,662 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-09-22 18:11:31,670 : INFO : collected 15658 word types and 72 unique tags from a corpus of 91 examples and 78890 words
2022-09-22 18:11:31,670 : INFO : Creating a fresh vocabulary
2022-09-22 18:11:31,681 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 6030 unique words (38.51% of original 15658, drops 9628)', 'datetime': '2022-09-22T18:11:31.681500', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 

In [9]:
# check word occurence
print(f"Word 'internet' appeared {model.wv.get_vecattr('internet', 'count')} times in the training corpus.")

Word 'internet' appeared 50 times in the training corpus.


In [10]:
model.train(trainCorpus, total_examples=model.corpus_count, epochs=model.epochs)

2022-09-22 18:11:31,724 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 6030 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-09-22T18:11:31.724055', 'gensim': '4.2.0', 'python': '3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]', 'platform': 'macOS-12.0.1-arm64-arm-64bit', 'event': 'train'}
2022-09-22 18:11:31,751 : INFO : EPOCH 0: training on 78890 raw words (53392 effective words) took 0.0s, 2012979 effective words/s
2022-09-22 18:11:31,779 : INFO : EPOCH 1: training on 78890 raw words (53206 effective words) took 0.0s, 2003254 effective words/s
2022-09-22 18:11:31,805 : INFO : EPOCH 2: training on 78890 raw words (53399 effective words) took 0.0s, 2076029 effective words/s
2022-09-22 18:11:31,832 : INFO : EPOCH 3: training on 78890 raw words (53209 effective words) took 0.0s, 2110230 effective words/s
2022-09-22 18:11:31,860 : INFO : EPOCH 4: training on 78

In [11]:
# infer a vector from corupus (I dont actually know what this means or does! :D )
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print("infering a vector:")
print(vector)

infering a vector:
[-0.04934597  0.00160057  0.26203486 -0.07383558 -0.0667066  -0.26820335
  0.43172204  0.39789876  0.14096175 -0.02815138 -0.42968407  0.0982025
 -0.36609375 -0.06926265  0.21068591  0.27722657 -0.32687858 -0.06640548
 -0.1935263  -0.09768109 -0.04143621 -0.02071577  0.2509165   0.37031558
  0.1908184   0.0456942  -0.0297021  -0.05964939 -0.11959986 -0.01323013
 -0.15883218 -0.04204777  0.20457451  0.0421306  -0.12221819  0.09911337
  0.11831775 -0.4438553   0.06492002 -0.4314228   0.28859678 -0.0255124
  0.03718105  0.19174854 -0.07558022  0.04398206 -0.3087246   0.0559519
  0.00469558  0.17997092]


In [16]:
# assessing the model
ranks = []
secondRanks = []
for doc_id in range(len(trainCorpus)):
        inferredVector = model.infer_vector(trainCorpus[doc_id].words)
        sims = model.dv.most_similar([inferredVector], topn=len(model.dv))
        rank = [docid for docid, sim in sims].index(doc_id)
        ranks.append(rank)
        print(doc_id)
        
        secondRanks.append(sims[1])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


ValueError: 72 is not in list

In [ ]:
print([docid for docid, sim in sims])